In [13]:
from pandas import *

from bokeh.plotting import figure
from bokeh.tile_providers import get_provider, CARTODBPOSITRON
from bokeh.io import output_notebook, show
from bokeh.models import LogColorMapper, ColumnDataSource
from bokeh.palettes import Oranges256 as oranges

from pyproj import Proj, transform

from sparql_dataframe import get
endpoint = 'https://dati.cultura.gov.it/sparql'

In [14]:
query = '''
SELECT DISTINCT(?institute) COUNT(DISTINCT(?s) as ?count)  ?lat ?long WHERE {
?s a ?o.
FILTER (?o = arco:CulturalProperty || ?o = arco:IntangibleCulturalProperty || ?o = arco:TangibleCulturalProperty || ?o = arco:ArchaeologicalProperty || ?o = arco:ImmovableCulturalProperty || ?o = arco:ArchitecturalOrLandscapeHeritage || ?o = arco:HistoricOrArtisticProperty || ?o = arco:MusicHeritage || ?o = arco:NaturalHeritage || ?o = arco:BotanicalHeritage || ?o = arco:MineralHeritage || ?o = arco:PalaeontologicalHeritage || ?o = arco:PertologicHeritage || ?o = arco:PlanetaryScienceHeritage || ?o = arco:ZoologicalHeritage || ?o = arco:NumismaticProperty || ?o = arco:PhotographicHeritage || ?o = arco:ScientificOrTechnologicalHeritage)
?s a-loc:hasCulturalInstituteOrSite ?institute.
  ?s clvapit:hasGeometry ?geometry .
  ?geometry a-loc:hasCoordinates ?coordinates .
?coordinates a-loc:lat ?lat.
?coordinates a-loc:long ?long.
}
'''
CP_coordinates = get(endpoint, query)

In [15]:
#since I have the institute for the properties I should keep the site instead of the subject
query = '''
SELECT DISTINCT(?s) ?dbpedia WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
?o cis:siteAddress ?address.
?address clvapit:hasCity ?urlcity.
?urlcity owl:sameAs ?urlcity2.
?urlcity2 owl:sameAs ?dbpedia.
FILTER (contains(str(?dbpedia), "dbpedia"))
}
'''

dbPediaLinks = get(endpoint, query)
dbPediaLinks

,s,dbpedia
0,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Besenello
1,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Calci
2,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Calci
3,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Calci
4,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Carmignano
...,...,...
21959,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Pavia
21960,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Pavia
21961,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Pavia
21962,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Pavia


In [16]:
dbpedia = dbPediaLinks["dbpedia"].drop_duplicates()
link = []

endpointDB = 'https://dbpedia.org/sparql'

CE_coordinates = DataFrame({"latitude":[], "longitude":[]})

for idx, item in dbpedia.items():
    query = '''
    SELECT ?latitude ?longitude WHERE {
       '''f'<{item}>''''  geo:lat ?latitude;
        geo:long ?longitude.
    } '''
    coordinates = get(endpointDB, query)
    link.append(item)
    CE_coordinates = concat([CE_coordinates, coordinates])

CE_coordinates.insert(0, "dbPedia", link)
CE_coordinates

http://dbpedia.org/resource/Besenello
   latitude  longitude
0   45.9333    11.1167
   latitude  longitude
0   45.9333    11.1167
http://dbpedia.org/resource/Calci
   latitude  longitude
0   43.7167    10.5167
   latitude  longitude
0   45.9333    11.1167
0   43.7167    10.5167
http://dbpedia.org/resource/Carmignano
   latitude  longitude
0   43.8167    11.0167
   latitude  longitude
0   45.9333    11.1167
0   43.7167    10.5167
0   43.8167    11.0167
http://dbpedia.org/resource/Cetona
   latitude  longitude
0   42.9667       11.9
   latitude  longitude
0   45.9333    11.1167
0   43.7167    10.5167
0   43.8167    11.0167
0   42.9667    11.9000
http://dbpedia.org/resource/Chiusi
   latitude  longitude
0   43.0167      11.95
   latitude  longitude
0   45.9333    11.1167
0   43.7167    10.5167
0   43.8167    11.0167
0   42.9667    11.9000
0   43.0167    11.9500
http://dbpedia.org/resource/Lucca
   latitude  longitude
0   43.8417    10.5028
   latitude  longitude
0   45.9333    11.1167
0  

,dbPedia,latitude,longitude
0,http://dbpedia.org/resource/Besenello,45.9333,11.11670
0,http://dbpedia.org/resource/Calci,43.7167,10.51670
0,http://dbpedia.org/resource/Carmignano,43.8167,11.01670
0,http://dbpedia.org/resource/Cetona,42.9667,11.90000
0,http://dbpedia.org/resource/Chiusi,43.0167,11.95000
...,...,...,...
0,http://dbpedia.org/resource/Casalciprano,41.5833,14.53330
0,http://dbpedia.org/resource/Monterosso_Grana,44.4167,7.31667
0,http://dbpedia.org/resource/Castrocielo,41.5333,13.70000
0,http://dbpedia.org/resource/Lu_e_Cuccaro_Monfe...,44.9972,8.47222


In [17]:
print(CE_coordinates)

                                              dbPedia  latitude  longitude
0               http://dbpedia.org/resource/Besenello   45.9333   11.11670
0                   http://dbpedia.org/resource/Calci   43.7167   10.51670
0              http://dbpedia.org/resource/Carmignano   43.8167   11.01670
0                  http://dbpedia.org/resource/Cetona   42.9667   11.90000
0                  http://dbpedia.org/resource/Chiusi   43.0167   11.95000
..                                                ...       ...        ...
0            http://dbpedia.org/resource/Casalciprano   41.5833   14.53330
0        http://dbpedia.org/resource/Monterosso_Grana   44.4167    7.31667
0             http://dbpedia.org/resource/Castrocielo   41.5333   13.70000
0   http://dbpedia.org/resource/Lu_e_Cuccaro_Monfe...   44.9972    8.47222
0                  http://dbpedia.org/resource/Sinnai   39.3000    9.20000

[794 rows x 3 columns]


In [18]:
CE_coordinates_cleaned = CE_coordinates.merge(dbPediaLinks, left_on="dbPedia", right_on="dbpedia")
CE_coordinates_cleaned

,dbPedia,latitude,longitude,s,dbpedia
0,http://dbpedia.org/resource/Besenello,45.9333,11.11670,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Besenello
1,http://dbpedia.org/resource/Calci,43.7167,10.51670,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Calci
2,http://dbpedia.org/resource/Calci,43.7167,10.51670,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Calci
3,http://dbpedia.org/resource/Calci,43.7167,10.51670,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Calci
4,http://dbpedia.org/resource/Calci,43.7167,10.51670,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Calci
...,...,...,...,...,...
21959,http://dbpedia.org/resource/Casalciprano,41.5833,14.53330,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Casalciprano
21960,http://dbpedia.org/resource/Monterosso_Grana,44.4167,7.31667,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Monterosso_Grana
21961,http://dbpedia.org/resource/Castrocielo,41.5333,13.70000,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Castrocielo
21962,http://dbpedia.org/resource/Lu_e_Cuccaro_Monfe...,44.9972,8.47222,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Lu_e_Cuccaro_Monfe...


In [22]:
CE_coordinates_cleaned = CE_coordinates_cleaned[["s", "latitude", "longitude"]].drop_duplicates(subset= ["s"])
CE_coordinates_cleaned.reset_index()

,index,s,latitude,longitude
0,0,http://dati.beniculturali.it/mibact/eventi/res...,45.9333,11.11670
1,1,http://dati.beniculturali.it/mibact/eventi/res...,43.7167,10.51670
2,2,http://dati.beniculturali.it/mibact/eventi/res...,43.7167,10.51670
3,3,http://dati.beniculturali.it/mibact/eventi/res...,43.7167,10.51670
4,4,http://dati.beniculturali.it/mibact/eventi/res...,43.7167,10.51670
...,...,...,...,...
20475,21958,http://dati.beniculturali.it/mibact/eventi/res...,44.5167,7.46667
20476,21959,http://dati.beniculturali.it/mibact/eventi/res...,41.5833,14.53330
20477,21960,http://dati.beniculturali.it/mibact/eventi/res...,44.4167,7.31667
20478,21962,http://dati.beniculturali.it/mibact/eventi/res...,44.9972,8.47222


In [20]:
CP_coordinates.to_csv("CP_coordinates.csv")
CE_coordinates_cleaned.to_csv("CE_coordinates.csv")